# **Assignment 3**

## **ECON8502 - Structural Microeconometrics**

### Conor Bayliss

#### **Setup**

In [1]:
using Optim, Distributions, Statistics, CSV, DataFrames, Plots, DataFramesMeta, LinearAlgebra

##### **Source code for the model**

In [10]:
include("C:\\Users\\bayle\\Documents\\Github\\micro_labour\\src\\model.jl")

plain_logit (generic function with 1 method)

##### **Setting up exogenous state variables**

In [11]:
struct model_data
    T::Int64 #<- length of problem

    y0::Int64 #<- year to begin problem
    age0::Int64 # <- mother's age at start of problem
    SOI::Vector{Int64} #<- state SOI in each year
    num_kids::Vector{Int64} #<- number of kids in household that are between age 0 and 17
    TotKids::Int64 #<- indicares the total number of children that the mother will have over the available panel
    age_kid::Matrix{Int64} #< age_kid[f,t] is the age of child f at time t. Will be negative if child not born yet.
    cpi::Vector{Float64} #<- cpi

    R::Vector{Int64} #<- indicates if work requirement in time t
    Kω::Int64 #<- indicates length of time limit once introduced
    TL::Vector{Bool} #<- indicating that time limit is in place
end

In [19]:
md = test_model()
p = pars(5,5) #<- set Kτ = 5 and Kε = 5

(Kτ = 5, Kε = 5, β = 0.98, αC = 1.0, αθ = [0.1, 0.1, 0.1, 0.1, 0.1], αH = [0.0, 0.0, 0.0, 0.0, 0.0], αA = [0.0, 0.0, 0.0, 0.0, 0.0], αS = [0.0, 0.0, 0.0, 0.0, 0.0], αR = [0.0, 0.0], σ = [2.0, 2.0], σ_W = 2.0, σε = 2.0, βΓx = [0.0, 0.0], βΓτ = [0.0, 0.0], βw = [6.0 6.375 … 7.125 7.5; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], βτ = [0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0; … ; 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0], πε = [0.2 0.2 … 0.2 0.2; 0.2 0.2 … 0.2 0.2; … ; 0.2 0.2 … 0.2 0.2; 0.2 0.2 … 0.2 0.2], εgrid = LinRange{Float64}(-1.0, 1.0, 5), αl = 1, πW = 0.8, ση = 2.0)

##### **Nested logit probabilities**

In [18]:
B₁ = [[1,2],[3,4],[5,6]]
C₁ = [[1,],[2,],[3,],[4,],[5,],[6,]]

B₂ = [[1,2,3]]
C₂ = [[1,2],[3,4],[5,6]]

B = (B₁,B₂)
C = (C₁,C₂)

([[1], [2], [3], [4], [5], [6]], [[1, 2], [3, 4], [5, 6]])

##### **Indexing the state**

In [20]:
# Hypothetical state space dimensions:
Kε = 5
Kτ = 5
Kω = 6

k_idx = LinearIndices((Kτ,Kε,Kω))
k_inv = CartesianIndices(k_idx)

# To get the aggregate index k, call:
k = k_idx[2,3,2]
@show k
# Then if we have k we can work back with:
k_tuple = Tuple(k_inv[k])
@show k_tuple

k = 37
k_tuple = (2, 3, 2)


(2, 3, 2)

##### **Transition probabilities**

In [15]:
fε(3,5,0.9)

((2, 3, 4), (0.04999999999999999, 0.9, 0.04999999999999999))

In [16]:
fε(1,5,0.9)

((1, 2), (0.04999999999999999, 0.95))

#### **Question 1**

Write a function `calc_vj` which calculates the choice-specific value (i.e. the deterministic value of the choice) of a particular choice $j$ in a particular time period $t$ given the state and other exogenous variables. If you are confident you can code this however you like, but given the existing setup you might like to write the function in a way that takes the following inputs:

* `j`: the discrete choice
* `t`: the time period in the model
* `state`: a tuple that contains the state $(k_{\tau}, k_{\epsilon}, k_{\omega})$ as well as a linear indexing rule
* `V`: a vector that contains the continuation value for each state at time `t+1`
* `pars`: the parameters of the model
* `md`: an instance of the `model_data` object that holds all relevant state variables

Verify that your function works by testing it on the `model_data` instance created by `test_model`. Use the `@time` macro to look at evaluation time and memory allocations.


include("utility.jl")


In [ ]:
function calc_vj(j,t,state,V,pars,md)
    u = model.utility()
end

#### **Question 2**

Write a function called `iterate!` that iterates over all states at time period $t$ and fills in choice probabilities and continuation values for period $t$ in pre-allocated arrays. Again, you can do this however you like but here is a suggested set of inputs:

* `t`: the time period
* `logP`: a $J$ x $K$ x $T$ array of choice probabilities where the function will fill in `logP[:,:,t]`
* `V`: a $K$ x $T$ array of continuation values
* `state_idx`: a named tuple that contains the size of the overall state space, a linear indexing rule that maps $(k_{\tau}, k_{\epsilon}, k_{\omega})$ to an overall state $k$, and a Cartesian Indexing rule that inverts the mapping
* `vj`: a $J$-dimensional vector that, for each state, can be used as a container for the choice-specific values
* `pars`: model parameters
* `md`: `model_data` for the problem

Verify that your function works by testing it on the `model_data` instance created by `test_model`. Use the `@time` macro to look at evaluation time and memory allocations.

#### **Question 3**

Write a function called `solve!` that performs backward induction to calculate continuation values and choice probabilities (storing them in pre-allocated arrays) in every period of the data across the whole state space. As before, some suggested inputs:

* `logP`: a $J$x$K$x$T$ array for the choice probabilities 
* `V`: a $K$ x $T$ array for continuation values
* `vj`: a container or buffer for choice-specific values in each iteration
* `pars`: model parameters
* `md`: an instance of `model_data`

Verify that your function works by testing it on the `model_data` instance created by `test_model`. Use the `@time` macro to look at evaluation time and memory allocations.